In [ ]:
import numpy as np
import pandas as pd
import time 
import random
from sklearn.model_selection import train_test_split
from sklearn import datasets, preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ds=pd.read_csv('/content/drive/MyDrive/diabetes.csv')
ds.head(15)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


# Import Data

In [ ]:

x = ds.drop(['Outcome'], axis = 1).values
y = ds['Outcome'].values
y = y.reshape(y.shape[0], 1)
#Spliting Data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=0)
#Standardize data
from sklearn.preprocessing import StandardScaler 
scale = StandardScaler()
x_train_std = scale.fit_transform(x_train)
x_test_std = scale.transform(x_test)


In [ ]:
print(y_train[0][0])

0


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

# Feed Forward

In [ ]:
def FF(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer

In [ ]:
NIL=8
HL=18
OL=1
#144+18+19=181

In [ ]:
def MLP(solution):
      fitness=0
      w=(NIL*HL)+(HL*OL)
      b=(NIL*HL)+HL+(HL*OL)+OL
      W=solution[0:w]
      B=solution[w:b]
      for idx,x in enumerate(x_train_std):
        actualValue=FF(HL,OL,W,B,x)

        fitness+=pow((actualValue[0]-y_train[idx][0]),2)

      fitness=fitness/len(x_train)
      
      return fitness

# WOA Algorithm

In [ ]:
import random
import numpy
import math
import time

class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0

def WOA(objf, lb, ub, dim, SearchAgents_no, Max_iter):

    # dim=30
    # SearchAgents_no=50
    # lb=-100
    # ub=100
    # Max_iter=500
    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim

    # initialize position vector and score for the leader
    Leader_pos = numpy.zeros(dim)
    Leader_score = float("inf")  # change this to -inf for maximization problems

    # Initialize the positions of search agents
    Positions = numpy.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = (
            numpy.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
        )

    # Initialize convergence
    convergence_curve = numpy.zeros(Max_iter)

    ############################
    s = solution()

    print('WOA is optimizing  "' + objf.__name__ + '"')

    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    ############################

    t = 0  # Loop counter

    # Main loop
    while t < Max_iter:
        for i in range(0, SearchAgents_no):

            # Return back the search agents that go beyond the boundaries of the search space

            # Positions[i,:]=checkBounds(Positions[i,:],lb,ub)
            for j in range(dim):
                Positions[i, j] = numpy.clip(Positions[i, j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness = objf(Positions[i, :])

            # Update the leader
            if fitness < Leader_score:  # Change this to > for maximization problem
                Leader_score = fitness
                # Update alpha
                Leader_pos = Positions[
                    i, :
                ].copy()  # copy current whale position into the leader position

        a = 2 - t * ((2) / Max_iter)
        # a decreases linearly fron 2 to 0 in Eq. (2.3)

        # a2 linearly decreases from -1 to -2 to calculate t in Eq. (3.12)
        a2 = -1 + t * ((-1) / Max_iter)

        # Update the Position of search agents
        for i in range(0, SearchAgents_no):
            r1 = random.random()  # r1 is a random number in [0,1]
            r2 = random.random()  # r2 is a random number in [0,1]

            A = 2 * a * r1 - a  # Eq. (2.3) in the paper
            C = 2 * r2  # Eq. (2.4) in the paper

            b = 1
            #  parameters in Eq. (2.5)
            l = (a2 - 1) * random.random() + 1  #  parameters in Eq. (2.5)

            p = random.random()  # p in Eq. (2.6)

            for j in range(0, dim):

                if p < 0.5:
                    if abs(A) >= 1:
                        rand_leader_index = math.floor(
                            SearchAgents_no * random.random()
                        )
                        X_rand = Positions[rand_leader_index, :]
                        D_X_rand = abs(C * X_rand[j] - Positions[i, j])
                        Positions[i, j] = X_rand[j] - A * D_X_rand

                    elif abs(A) < 1:
                        D_Leader = abs(C * Leader_pos[j] - Positions[i, j])
                        Positions[i, j] = Leader_pos[j] - A * D_Leader

                elif p >= 0.5:

                    distance2Leader = abs(Leader_pos[j] - Positions[i, j])
                    # Eq. (2.5)
                    Positions[i, j] = (
                        distance2Leader * math.exp(b * l) * math.cos(l * 2 * math.pi)
                        + Leader_pos[j]
                    )

        convergence_curve[t] = Leader_score
        if t % 9 == 0:
            print(
                ["At iteration " + str(t) + " the best fitness is " + str(Leader_score)]
            )
        t = t + 1

    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = convergence_curve
    s.optimizer = "WOA"
    s.objfname = objf.__name__
    s.best = Leader_score
    s.bestIndividual = Leader_pos

    return s

In [ ]:
A=WOA(MLP, -1, 1, 181, 50, 1000)


WOA is optimizing  "MLP"
['At iteration 0 the best fitness is 0.20376215400745848']
['At iteration 9 the best fitness is 0.20235886959162508']
['At iteration 18 the best fitness is 0.19992288934603689']
['At iteration 27 the best fitness is 0.19746931650768673']
['At iteration 36 the best fitness is 0.19559282394597852']
['At iteration 45 the best fitness is 0.19118194545398637']
['At iteration 54 the best fitness is 0.1866321478878947']
['At iteration 63 the best fitness is 0.1830086256488593']
['At iteration 72 the best fitness is 0.1813147446685203']
['At iteration 81 the best fitness is 0.17967011846092568']
['At iteration 90 the best fitness is 0.17931427051279877']
['At iteration 99 the best fitness is 0.17592970685134038']
['At iteration 108 the best fitness is 0.17559571196252233']
['At iteration 117 the best fitness is 0.1751610280332206']
['At iteration 126 the best fitness is 0.17468855191758603']
['At iteration 135 the best fitness is 0.17387977192865736']
['At iteration 14

In [ ]:
print(A.convergence[999])

0.15920959092033676


In [ ]:
solution=A.bestIndividual
print(solution)

[-1.         -1.          0.06132637 -0.19659421  0.67150907 -1.
 -0.25648339 -1.          0.61017644 -0.70058199  0.41900677  0.0064356
 -1.          0.13252206 -0.35362439 -1.         -1.         -1.
  1.         -1.          1.         -0.04498099  0.57246531 -0.7698138
 -0.0078449  -1.         -0.07395135  0.01857883 -1.          1.
 -1.          0.80674214 -1.          1.         -1.         -1.
  0.2660233  -0.55691883 -0.1108763  -0.50784266  0.56192576 -1.
 -0.4027023  -0.6408892   0.1821014   0.64689863 -1.         -1.
  0.15700527 -0.95210525  0.30704861 -1.          0.01934495  0.81702029
 -0.05357514 -1.         -1.         -1.         -1.         -0.18081262
 -0.03541313  0.52239455  0.22046293 -1.         -0.01081504 -0.03321494
 -1.         -1.          0.07266499 -1.          0.03205881 -0.07562414
 -0.31118723 -1.          0.74386553 -1.         -0.07632097 -0.64237125
  1.         -0.1499842  -1.         -0.18529192 -1.          1.
 -1.         -0.10861878 -1.        

In [ ]:
def Test(HL,OL,W,B,x):
  INL=len(x)
  hidden_layer=np.zeros((HL))
  output_layer=np.zeros((OL))
  witr=0
  bitr=0
  for i in range(HL):
    sum=0
    for j in range(INL):
      sum=sum+(x[j]*W[witr])
      witr=witr+1
    sum=sum.sum()
    hidden_layer[i]=sigmoid(sum+B[bitr])
    bitr=bitr+1

  for i in range(OL):
    output=0
    for j in range(HL):
      output=output+(hidden_layer[j]*W[witr])
      witr = witr + 1
    output=output.sum()
    output_layer[i]=sigmoid(output+B[bitr])
    bitr=bitr+1
  return output_layer




In [ ]:
min_key=min(B.keys(), key=(lambda k: B[k]))

In [ ]:
w=(NIL*HL)+(HL*OL)
b=(NIL*HL)+HL+(HL*OL)+OL    
W=solution[0:w]
B=solution[w:b]
print(len(W))
print(B)

162
[-1.          0.69723143  1.         -0.15862736 -0.27194478 -1.
 -1.         -0.10851927 -0.30737248 -0.02217668 -0.13397315 -0.49631705
 -0.2859298  -1.          0.18403168 -1.         -1.         -0.43300004
  1.        ]


In [ ]:
a=Test(HL,OL,W,B,x_test_std[7])
print(a)


[1.]


In [ ]:
R=0
for i in range(len(y_train)):
  p=Test(HL,OL,W,B,x_train_std[i])
  if y_train[i][0]==0:
    if p[0]<0.5:
      R+=1
  if y_train[i][0]==1:
    if p[0]>0.5:
      R+=1


In [ ]:
print((R*100)/len(y_train))

76.90972222222223


In [ ]:
def prediction(HL,OL,W,B,x):
  predict=[]
  s=[]
  for i in range(len(x)):
    a=Test(HL,OL,W,B,x[i])
    if a[0]>0.5:
      predict.append(1)
    else:
      predict.append(0)
    
    
  
  return predict

In [ ]:
pred=prediction(HL,OL,W,B,x_test_std)
print(f"Predict :{pred}")
y_true = sum(y_test.tolist(), [])
print(f"true y:{y_true}")

acc=0
for i in range(len(y_test)):
  if pred[i]==y_true[i]:
    acc+=1

print(f"accuracy:{acc/len(y_test)*100}%")

Predict :[1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0]
true y:[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1

In [ ]:
p=[[2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173583065964e-08], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859951413703e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999997594005218, 1.7827926890587977e-14], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3777059110728068e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3441849876730426e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [2.3443797239678989e-07, 1.0], [0.999997046667274, 1.0], [3.4230559529451456e-19, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.7353786067445575e-13, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998619, 7.412144614465116e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [3.2958383525020503e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537748567615e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [3.295837933196582e-19, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.735378606784197e-13, 1.0], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [4.7601859963200897e-07, 1.0], [1.0, 7.49479046619701e-09], [2.730081969202577e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [2.824682922217676e-13, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.494780457447032e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.2482862984116327e-18, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.494790466200846e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.7601836872318313e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999999998, 3.6456980727045307e-10], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [3.561895408985392e-19, 1.0], [0.9999999999998617, 7.413141488936635e-08], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.999997046667274, 1.0], [1.0, 7.49479046619701e-09], [2.3443797226534723e-07, 1.0], [0.9999999999997193, 1.377389407325726e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [2.735371990190059e-13, 1.0], [1.0, 7.49479046619701e-09], [3.295837933196582e-19, 1.0], [1.0, 5.790537731467655e-12], [0.9999999954925718, 1.1219721194964509e-13], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.735378606754956e-13, 1.0], [2.344379723967903e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.7601861964494985e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 1.4646590413208634e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.416891915703537e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [4.7601859947178284e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997216, 1.382760290185998e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 7.494787237171255e-09], [0.9999997593963287, 1.7827783200580683e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [1.0, 5.790537731467655e-12], [5.186223429698876e-06, 1.0], [1.0, 7.49479046619701e-09], [3.2963276408309863e-19, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [4.760185995141387e-07, 1.0], [0.999999759396326, 1.7827783109247703e-14], [2.3443797239678989e-07, 1.0], [2.3443797239533066e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [3.539949630789169e-14, 1.0], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071096696e-17], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.735378606784197e-13, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.399724176787439e-09], [3.2958381859094635e-19, 1.0], [1.0, 5.790537731467655e-12], [2.3443797239678989e-07, 1.0], [2.736071297671768e-13, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998617, 7.413173838927611e-08], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.494790466196905e-09], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [1.0, 5.790537731467655e-12], [1.0, 7.49479046619701e-09], [1.0, 7.49479046619701e-09], [0.9999997597973544, 1.784154246233934e-14], [4.760185995141387e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773916445774456e-17], [1.0, 5.7905377175175346e-12], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105640535e-14], [1.0, 7.49479046619701e-09], [0.9999999999997193, 1.3773894071084169e-17], [4.760185994289154e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 5.790537731467655e-12], [0.9999999999997193, 1.3773894071084169e-17], [0.9999997594030715, 1.782801427057929e-14], [0.9999999999997193, 1.3773894071084169e-17], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071148567e-17], [2.3443797239678989e-07, 1.0], [4.760185995141387e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999999999998492, 1.8358616739546913e-17], [2.3443797239678989e-07, 1.0], [2.3443792937421956e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [2.735378606784197e-13, 1.0], [1.0, 7.49479046619701e-09], [2.3443797239678989e-07, 1.0], [1.0, 5.790537731467655e-12], [1.0, 7.494790466200632e-09], [0.9999997593963258, 1.7827783105267858e-14], [0.9999997593963258, 1.7827783105267858e-14], [0.9999999999997193, 1.3773894071084169e-17], [0.9999999999997193, 1.3773894071086518e-17], [2.3443797239678989e-07, 1.0], [0.9999999999997193, 1.3773894071084169e-17], [2.8247268646226163e-13, 1.0], [4.760185922590398e-07, 1.0], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 7.493736721731287e-09], [4.760185995141387e-07, 1.0], [2.3443797239678989e-07, 1.0], [4.760185995142792e-07, 1.0], [1.0, 7.49479046619701e-09], [0.9999997593963258, 1.7827783105267858e-14], [1.0, 5.790537731467655e-12], [3.295837933196582e-19, 1.0], [1.0, 5.7905377314674905e-12], [1.0, 5.790539620623675e-12], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.492433652437726e-09], [2.3443797239678989e-07, 1.0], [2.3443797239678989e-07, 1.0], [1.0, 7.49479046619701e-09], [4.760185995141387e-07, 1.0], [4.760185995141387e-07, 1.0], [5.0873878446689e-18, 1.0]]
for i in range(len(pred)):
  if p[i][0] and p[i][1] >0.5:
    print(i)
  